### data analysis stage:
#### 1. Join the data
#### 2. Research ways to analyse how each of the features affects the probability of a transaction to be fraud

In [2]:
import pandas as pd

df_train = pd.read_csv('dataset/model_dataset/transactions_train.csv')
df_test = pd.read_csv('dataset/model_dataset/transactions_test.csv')
column_name = 'IS_RECURRING_TRANSACTION'

def clean_column(value):
    if value is True:
        return "True"
    elif (value is False) or (value=="Fals"):
        return "False"
    return str(value)

# Apply the function to the column
df_train[column_name] = df_train[column_name].apply(clean_column)
df_test[column_name] = df_test[column_name].apply(clean_column)

df_train.to_csv('preprocessed_train.csv', index=False)
df_test.to_csv('preprocessed_test.csv', index=False)

print(f"Column '{column_name}' cleaned and saved successfully!")

C:\Users\bbdnet2520\AppData\Local\Temp\ipykernel_31928\119424248.py:3: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('dataset/model_dataset/transactions_train.csv')


Column 'IS_RECURRING_TRANSACTION' cleaned and saved successfully!


In [ ]:
import numpy as np

df = pd.read_csv('dataset/model_dataset/clean_transactions_train.csv')

df['CUST_TO_TERM_DIST'] = np.sqrt((df['x_terminal_id'] - df['x_customer_id']) ** 2 +
                                  (df['y_terminal__id'] - df['y_customer_id']) ** 2)
df.drop(columns=['x_customer_id', 'y_customer_id', 'x_terminal_id', 'y_terminal__id', 'CUSTOMER_ID', 'TERMINAL_ID'], inplace=True)
df.to_csv('cleaned_file_with_distances.csv', index=False)


In [ ]:
import pandas as pd

df_train = pd.read_csv('dataset/model_dataset/transactions_train.csv')
df_test = pd.read_csv('dataset/model_dataset/transactions_test.csv')
column_name = 'IS_RECURRING_TRANSACTION'

def clean_column(value):
    if value is True:
        return "True"
    elif (value is False) or (value=="Fals"):
        return "False"
    return str(value)

# Apply the function to the column
df_train[column_name] = df_train[column_name].apply(clean_column)
df_test[column_name] = df_test[column_name].apply(clean_column)

df_train.to_csv('preprocessed_train.csv', index=False)
df_train.to_csv('preprocessed_test.csv', index=False)

print(f"Column '{column_name}' cleaned and saved successfully!")

In [11]:
import pandas as pd

df_train = pd.read_csv('preprocessed_train.csv', index_col='TX_ID')
df_test = pd.read_csv('preprocessed_test.csv', index_col='TX_ID')
#print(df_train.columns)  # Check if the new column exists in the saved CSV
#print(df[['CUST_TO_TERM_DIST']].head())
#full_t_train = pd.read_csv('dataset/model_dataset/full_transactions_train.csv', index_col='TX_ID')
#full_t_test = pd.read_csv('dataset/model_dataset/full_transactions_test.csv', index_col='TX_ID')
#transactions_train = pd.read_csv('dataset/model_dataset/transactions_train.csv')
#transactions_test = pd.read_csv('dataset/model_dataset/transactions_test.csv')
customers = pd.read_csv('dataset/model_dataset/customers.csv', index_col='CUSTOMER_ID')
merchants = pd.read_csv('dataset/model_dataset/merchants.csv', index_col='MERCHANT_ID')
terminals = pd.read_csv('dataset/model_dataset/terminals.csv', index_col='TERMINAL_ID')
##Merge transactions with customers on 'customer_id'
transactions_with_customers = pd.merge(df_train, customers, on='CUSTOMER_ID', how='left')
transactions_with_customers_terminals = pd.merge(transactions_with_customers, terminals, on='TERMINAL_ID', how='left')
full_t = pd.merge(transactions_with_customers_terminals, merchants, on='MERCHANT_ID', how='left')
#print(full_transactions.head())
#full_t.to_csv('full_transactions_train.csv', index=False)
full_t.columns


Index(['TX_TS', 'CUSTOMER_ID', 'TERMINAL_ID', 'TX_AMOUNT', 'TX_FRAUD',
       'TRANSACTION_GOODS_AND_SERVICES_AMOUNT', 'TRANSACTION_CASHBACK_AMOUNT',
       'CARD_EXPIRY_DATE', 'CARD_DATA', 'CARD_BRAND', 'TRANSACTION_TYPE',
       'TRANSACTION_STATUS', 'FAILURE_CODE', 'FAILURE_REASON',
       'TRANSACTION_CURRENCY', 'CARD_COUNTRY_CODE', 'MERCHANT_ID',
       'IS_RECURRING_TRANSACTION', 'ACQUIRER_ID', 'CARDHOLDER_AUTH_METHOD',
       'x_customer_id', 'y_customer_id', 'x_terminal_id', 'y_terminal__id',
       'BUSINESS_TYPE', 'MCC_CODE', 'LEGAL_NAME', 'FOUNDATION_DATE',
       'TAX_EXCEMPT_INDICATOR', 'OUTLET_TYPE', 'ACTIVE_FROM', 'TRADING_FROM',
       'ANNUAL_TURNOVER_CARD', 'ANNUAL_TURNOVER', 'AVERAGE_TICKET_SALE_AMOUNT',
       'PAYMENT_PERCENTAGE_FACE_TO_FACE', 'PAYMENT_PERCENTAGE_ECOM',
       'PAYMENT_PERCENTAGE_MOTO', 'DEPOSIT_REQUIRED_PERCENTAGE',
       'DEPOSIT_PERCENTAGE', 'DELIVERY_SAME_DAYS_PERCENTAGE',
       'DELIVERY_WEEK_ONE_PERCENTAGE', 'DELIVERY_WEEK_TWO_PERCENTAGE',


In [14]:
cols_missing = [col for col in full_t.columns if full_t[col].isnull().any()]
print(cols_missing)

[]


In [13]:
from sklearn.model_selection import train_test_split

# Remove rows with missing target, separate target from predictors
full_t.dropna(axis=0, subset=['TX_FRAUD'], inplace=True)
y = full_t.TX_FRAUD
full_t.drop(['TX_FRAUD'], axis=1, inplace=True)

# To keep things simple, we'll drop columns with missing values
full_t.drop(cols_missing, axis=1, inplace=True)
print(full_t.columns)

drop_cols = [
  'TX_TS',
  'CUSTOMER_ID',
  'TERMINAL_ID',
  'CARD_DATA',
  'CARD_EXPIRY_DATE',
  'MERCHANT_ID', 'MCC_CODE',
  'LEGAL_NAME',
  'FOUNDATION_DATE',
  'ACTIVE_FROM',
  'TRADING_FROM'
] # drop columns that cannot be encoded
full_t.drop(drop_cols, axis=1, inplace=True)
print(full_t.columns)

# Break off validation set from training data
#X_train, X_valid, y_train, y_valid = train_test_split(df, y, train_size=0.8, test_size=0.2, random_state=0)

Index(['TX_TS', 'CUSTOMER_ID', 'TERMINAL_ID', 'TX_AMOUNT',
       'TRANSACTION_GOODS_AND_SERVICES_AMOUNT', 'TRANSACTION_CASHBACK_AMOUNT',
       'CARD_EXPIRY_DATE', 'CARD_DATA', 'CARD_BRAND', 'TRANSACTION_TYPE',
       'TRANSACTION_STATUS', 'TRANSACTION_CURRENCY', 'CARD_COUNTRY_CODE',
       'MERCHANT_ID', 'IS_RECURRING_TRANSACTION', 'x_customer_id',
       'y_customer_id', 'x_terminal_id', 'y_terminal__id', 'BUSINESS_TYPE',
       'MCC_CODE', 'LEGAL_NAME', 'FOUNDATION_DATE', 'TAX_EXCEMPT_INDICATOR',
       'OUTLET_TYPE', 'ACTIVE_FROM', 'TRADING_FROM', 'ANNUAL_TURNOVER_CARD',
       'ANNUAL_TURNOVER', 'AVERAGE_TICKET_SALE_AMOUNT',
       'PAYMENT_PERCENTAGE_FACE_TO_FACE', 'PAYMENT_PERCENTAGE_ECOM',
       'PAYMENT_PERCENTAGE_MOTO', 'DEPOSIT_REQUIRED_PERCENTAGE',
       'DEPOSIT_PERCENTAGE', 'DELIVERY_SAME_DAYS_PERCENTAGE',
       'DELIVERY_WEEK_ONE_PERCENTAGE', 'DELIVERY_WEEK_TWO_PERCENTAGE',
       'DELIVERY_OVER_TWO_WEEKS_PERCENTAGE'],
      dtype='object')
Index(['TX_AMOUNT', 'TRANS

In [ ]:
#test_cols = ['TX_AMOUNT', 'TRANSACTION_GOODS_AND_SERVICES_AMOUNT','TRANSACTION_CASHBACK_AMOUNT', 'CARD_BRAND', 'TRANSACTION_TYPE',
#          'TRANSACTION_STATUS', 'TRANSACTION_CURRENCY', 'CARD_COUNTRY_CODE','IS_RECURRING_TRANSACTION']
#
#X_test = df[test_cols]
#
#drop_X_test = X_test.select_dtypes(exclude=['object'])
#print(drop_X_test.columns)

drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])
print(drop_X_train.columns)
print(drop_X_valid.columns)

In [ ]:
#print(X_train.select_dtypes(include=['object']).columns)

# ts_tx can't use ordinal/one-hot encoding
# card_expiry_date can't use ordninal/one-hot encoding
# merchant_id can't use ordninal/one-hot encoding

cols = [
  'CARD_BRAND',
  'TRANSACTION_TYPE',
  'TRANSACTION_STATUS',
  'TRANSACTION_CURRENCY',
  'CARD_COUNTRY_CODE',
  'IS_RECURRING_TRANSACTION',

] # ordinal encoded columns
#enc_cols = ['CARD_BRAND', 'TRANSACTION_TYPE', 'TRANSACTION_STATUS','TRANSACTION_CURRENCY', 'CARD_COUNTRY_CODE']

for col in cols:
  print(X_test[col].unique())

#for col in cols:
#  print(X_train[col].unique())
#  print(X_valid[col].unique())
#  print()

In [ ]:
# Categorical columns in the training data
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]
print(object_cols)
print()

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if
                   set(X_valid[col]).issubset(set(X_train[col]))]

# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(cols))
bad_label_cols2 = list(set(good_label_cols)-set(cols))

#bad = ['CARD_EXPIRY_DATE', 'FOUNDATION_DATE' ]

print('Categorical columns that will be ordinal encoded:', cols)
#print('Columns that I checked:', cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols2)

In [23]:
from sklearn.preprocessing import OrdinalEncoder

# Apply ordinal encoder
ordinal_encoder = OrdinalEncoder()
X_train[cols] = ordinal_encoder.fit_transform(X_train[cols])
X_valid[cols] = ordinal_encoder.transform(X_valid[cols])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

print("MAE from Approach 2 (Ordinal Encoding):")

model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(mean_absolute_error(y_valid, preds))

In [ ]:
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

In [61]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_test = pd.DataFrame(OH_encoder.fit_transform(X_test[cols]))
#OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_test[cols]))

# One-hot encoding removed index; put it back
OH_cols_test.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_test = X_test.drop(object_cols, axis=1)
#num_X_valid = X_test.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

# Ensure all columns have string type
OH_X_test.columns = OH_X_test.columns.astype(str)

## Use as many lines of code as you need!
# Apply one-hot encoder to each column with categorical data
#OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
#OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[cols]))
#OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[cols]))
#
## One-hot encoding removed index; put it back
#OH_cols_train.index = X_train.index
#OH_cols_valid.index = X_valid.index
#
## Remove categorical columns (will replace with one-hot encoding)
#num_X_train = X_train.drop(object_cols, axis=1)
#num_X_valid = X_valid.drop(object_cols, axis=1)
#
## Add one-hot encoded columns to numerical features
#OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
#OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)
#
## Ensure all columns have string type
#OH_X_train.columns = OH_X_train.columns.astype(str)
#OH_X_valid.columns = OH_X_valid.columns.astype(str)


In [ ]:
print("MAE from Approach 3 (One-hot Encoding):")

model = RandomForestRegressor(n_estimators=100, random_state=0)
#model.fit(OH_X_train, y_train)
#preds = model.predict(OH_X_valid)
#print(mean_absolute_error(y_valid, preds))

In [ ]:
model.fit(OH_X_train, y_train)
preds = model.predict(OH_X_valid)
print(mean_absolute_error(y_valid, preds))